In [6]:
# Standard Library
import asyncio
import os

# Third-Party Libraries
import faiss
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

# Langchain
from langchain.text_splitter import MarkdownTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Semantic Kernel
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.functions.kernel_arguments import KernelArguments

# Event Loop Patch for Jupyter
import nest_asyncio

# Load environment variables
load_dotenv()

search_api = os.getenv("AI_SEARCH_API")
search_endpoint = "https://cardassist.search.windows.net"
ai_foundry_api = os.getenv("AI_FOUNDRY_MODEL_API")
llm_endpoint = os.getenv("LLM_ENDPOINT")

# Patch the event loop for Jupyter compatibility
nest_asyncio.apply()

pdf_doc_path = "./global_card_access_user_guide.pdf"
loader = PyPDFLoader(pdf_doc_path)
documents = loader.load()
text_splitter = MarkdownTextSplitter(chunk_size=300, chunk_overlap=30)
md_docs = text_splitter.split_documents(documents)
md_docs = [doc.page_content for doc in md_docs]


# Initialize embedding model
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

embedding_service = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device, model_kwargs={"trust_remote_code": True})

# Generate and index embeddings
embedding_matrix = np.array(
    [embedding_service.encode([doc])[0] for doc in tqdm(md_docs, desc="Generating embeddings")]
).astype("float32")

index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

Generating embeddings: 100%|██████████| 161/161 [00:01<00:00, 123.97it/s]


## Creating plugins for Activate, De-activate credit card and RAG QA 

In [7]:
from typing import Annotated
from semantic_kernel.functions import kernel_function

class CreditCardPlugin:
    @kernel_function(
        description="Deactivate a credit card; returns a confirmation message"
    )
    async def deactivate_card(self, card_number: Annotated[str, "The credit card number to deactivate"]) -> str:
        print("Function called to deactivate card")
        return f"Credit card {card_number} has been deactivated."

    @kernel_function(
        description="Activate a credit card; returns a confirmation message"
    )
    async def activate_card(self, card_number: Annotated[str, "The credit card number to activate"]) -> str:
        print("Function called to activate card")
        return f"Credit card {card_number} has been activated."
    
    @kernel_function(
        description="Get card information and other general information about the card and account management"
    )
    async def rag_query(self, query: Annotated[str, "The user query for RAG (Retrieval-Augmented Generation)"]):
        print("Function called for RAG query")
        query_embedding = embedding_service.encode([query])
        query_embedding = np.array(query_embedding).astype("float32")

        k = 10
        distances, indices = index.search(query_embedding, k)
        relevant_chunks = [md_docs[i] for i in indices[0]]

        context = "\n".join(relevant_chunks)
        augmented_prompt = f"{context}\n\nUser Query: {query}"
        
        return augmented_prompt

## Initializing Agent


In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins.time_plugin import TimePlugin

# Initialize the kernel
kernel = Kernel()

model_id = "gpt-4o-mini"

# Add the Azure OpenAI chat completion service
kernel.add_service(
    AzureChatCompletion(deployment_name=model_id,
                        endpoint=llm_endpoint,
                        api_key=ai_foundry_api)
)

# Add a plugin
kernel.add_plugin(
    CreditCardPlugin(),
    plugin_name="CreditCard",
)

arguments = KernelArguments(
    settings=AzureChatPromptExecutionSettings(
        # Advertise all functions from the WeatherPlugin, DateTimePlugin, and LocationPlugin plugins to the AI model.
        function_choice_behavior=FunctionChoiceBehavior.Auto(),
        # function_choice_behavior=FunctionChoiceBehavior.Required(filters={"included_functions": ["deactivate_card", "activate_card"]}),
        top_p= 0.9,
        temperature=0,
    )
)

async def run_model(user_query: str , system_prompt: str ):
    resp = await kernel.invoke_prompt(prompt = f"{system_prompt}\n\nUser Query: {user_query}", arguments=arguments )
    return resp.value

In [9]:
system_prompt = "You're a helpful assistant that can answer questions about credit card management, including activating and deactivating cards, and providing information about card features and account management. Make sure you provide accurate and well structured information based on the provided context."

print(system_prompt)

You're a helpful assistant that can answer questions about credit card management, including activating and deactivating cards, and providing information about card features and account management. Make sure you provide accurate and well structured information based on the provided context.


## Testing card activation

In [10]:
user_query = "Can you activate my credit card 1234-5678-9012-3456?"

# Run the asynchronous function
response = asyncio.run(run_model(system_prompt=system_prompt, user_query=user_query))
print("\nResponse:\n", response[0].content)

Function called to activate card

Response:
 Your credit card **1234-5678-9012-3456** has been successfully activated. If you have any other questions or need further assistance, feel free to ask!


## Testing card deactivation

In [11]:
user_query = "Can you deactivate my credit card 1234-5678-9012-3456?"

# Run the asynchronous function
response = asyncio.run(run_model(system_prompt=system_prompt, user_query=user_query))
print("\nResponse:\n", response[0].content)

Function called to deactivate card

Response:
 Your credit card ending in 3456 has been successfully deactivated. If you need any further assistance, feel free to ask!


## Testing QA RAG

In [12]:
user_query = "Steps for First-time Registration for Corporate Accounts"

# Run the asynchronous function
response = asyncio.run(run_model(system_prompt=system_prompt, user_query=user_query))
print("\nResponse:\n", response[0].content)

Function called for RAG query

Response:
 To register as a new user for a corporate account, follow these steps:

1. **Access the Global Card Access Website**: Go to [Global Card Access](https://www.bankofamerica.com/globalcardaccess).

2. **Initiate Registration**: On the Global Card Access sign-in screen, click on "Register now." 
   - If your organization has multiple corporate accounts, a "Select Corporate Account(s)" window will appear. Choose the appropriate corporate account and click "OK."

3. **Create Account Request Key(s)**: 
   - Complete the "Create Account Request Key" page. 
   - Configure your Account Request Key in the Settings section. This key will be used by your employees to request accounts online.

4. **Add Corporate Accounts**: 
   - To add a corporate account, click "Add" from the bottom left-hand side of the Corporate Accounts screen. Review and approve new account requests. Note that your company must be configured for this feature.

5. **Role Assignment**: 


## Hybrid query

In [13]:
user_query = "Can you activate my card 1234-5678-9012-3456? Also, let me know it's benefits and features."

# Run the asynchronous function
response = asyncio.run(run_model(system_prompt=system_prompt, user_query=user_query))
print("\nResponse:\n", response[0].content)

Function called to activate card
Function called for RAG query

Response:
 Your credit card **1234-5678-9012-3456** has been successfully activated.

### Benefits and Features of Your Card:
1. **Online Card Management**: Access to Global Card Access, an online tool that allows you to manage your card, check your credit limit, balance, and available credit.
2. **Security Features**: Options to lock your card, manage alerts, and change your PIN for added security.
3. **Convenient Payments**: Ability to make payments online (available for North America accounts).
4. **Access to Statements**: View and download your statements for better financial tracking.
5. **Customizable Account Settings**: Manage settings related to your account, including linking to corporate accounts and card types.

If you have any more questions or need further assistance, feel free to ask!
